In [34]:
# Basic python packages
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict
import glob
import splitfolders
import cv2
import random

# General machine learning packages
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Packages related to images
from PIL import Image
import PIL

# Packages for neural networks
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Embedding
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image

In [35]:
# !pip install --upgrade tensorflow_hub

In [36]:
# Check if GPU works
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [37]:
def clean_data(path):
    df = pd.read_csv(path)
    df = df[df['Creator'] != ''] 
    df = df[df['Creator'] != 'anoniem']
    df = df[df['Creator'] != 'onbekend']
    df = df[df['Creator'].notna()]
    df = df.replace('Koning, Cornelis (?-1671)', 'Koning, Cornelis')
    return df

def preprocess_data(df, num_artworks = 10):
    df = df.groupby("Creator").filter(lambda x: len(x) >= num_artworks)
    le = preprocessing.LabelEncoder()
    le.fit(df.Creator)
    df['Creator_cat'] = le.transform(df.Creator)
    le.fit(df.Type)
    df['Type_cat'] = le.transform(df.Type)
    return df

df = clean_data('../Data/Rijksmuseum/xml_files.csv')
rijksdata = preprocess_data(df, 500)

In [38]:
rijksdata

,Identifier,Creator,Title,Date,Type,Creator_cat,Type_cat
100,RP-T-1905-158(R),"Fokke, Simon","Jonge edelman, naar links in gebogen houding",1710 - 1779,tekening,5,51
101,RP-T-1905-158(V),"Fokke, Simon",Zittende man,ca. 1710 - ca. 1779,tekening,5,51
2275,SK-C-5,Rembrandt Harmensz. van Rijn,Officieren en andere schutters van wijk II in ...,1642 - 1642,schilderij,18,38
2276,SK-C-6,Rembrandt Harmensz. van Rijn,De Staalmeesters: het college van staalmeester...,1662 - 1662,schilderij,18,38
2277,SK-A-3340,Rembrandt Harmensz. van Rijn,Borstbeeld van een man in oosterse kleding (),1635 - 1635,schilderij,18,38
...,...,...,...,...,...,...,...
111901,RP-P-OB-85.858,"Vinkeles, Reinier",Plunderen van huizen van patriotten te Devente...,1787 - 1795,prent,20,35
111902,RP-P-OB-85.859,"Vinkeles, Reinier",Plunderen van huizen van patriotten te Devente...,1787 - 1795,prent,20,35
111903,RP-P-OB-85.860,"Vinkeles, Reinier","Vlucht van patriotten uit Franeker, 1787 ()",1787 - 1795,prent,20,35
111947,BK-17474,Meissener Porzellan Manufaktur,Servies,ca. 1750 - ca. 1760,servies,15,40


In [44]:
#Paths to different folders/files
image_dir = "../Data/Rijksmuseum/jpg2/"
split_image_dir = "../Data/Rijksmuseum/jpg2_split_500/"
training_path = "../Data/Rijksmuseum/output_500/train/"
validation_path = "../Data/Rijksmuseum/output_500/val/"
testing_path = "../Data/Rijksmuseum/output_500/test/"

img_size = (200, 200) #Size of the input of the neural networks
IMG_SHAPE = img_size + (3,)
batch_size = 256
n_labels_c1 = len(rijksdata.groupby('Creator').count())
n_labels_c2 = len(rijksdata.groupby('Type').count())

In [45]:
labels = rijksdata[['Identifier', 'Creator_cat', 'Type_cat']]
sorted_list = sorted(list(labels.Type_cat.unique()))
sorted_list
# labels[labels['Identifier'] == 'RP-P-1878-A-550']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65]

In [46]:
print(n_labels_c1, n_labels_c2)

21 66


In [70]:
train_dict = {}
val_dict = {}
test_dict = {}

for fol in os.listdir(training_path):
    for file in os.listdir(os.path.join(training_path, fol)):
        im_path = os.path.join(os.path.join(training_path, fol), file)
        train_dict[im_path] = (labels[labels['Identifier'] == file[:-4]]['Creator_cat'].values[0], labels[labels['Identifier'] == file[:-4]]['Type_cat'].values[0])
        
for fol in os.listdir(validation_path):
    for file in os.listdir(os.path.join(validation_path, fol)):
        im_path = os.path.join(os.path.join(validation_path, fol), file)
        val_dict[im_path] = (labels[labels['Identifier'] == file[:-4]]['Creator_cat'].values[0], labels[labels['Identifier'] == file[:-4]]['Type_cat'].values[0])
        
for fol in os.listdir(testing_path):
    for file in os.listdir(os.path.join(testing_path, fol)):
        im_path = os.path.join(os.path.join(testing_path, fol), file)
        test_dict[im_path] = (labels[labels['Identifier'] == file[:-4]]['Creator_cat'].values[0], labels[labels['Identifier'] == file[:-4]]['Type_cat'].values[0])

In [48]:
def create_model_inception(c_1, c_2):
    inputs = tf.keras.layers.Input(shape=[img_size[0], img_size[1], 3], name='main_input')
    main_branch = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/inception_v3/classification/4")(inputs)
    main_branch = tf.keras.layers.Flatten()(main_branch)
    main_branch = tf.keras.layers.Dense(1024, activation='relu')(main_branch)
    main_branch = tf.keras.layers.Dropout(0.5)(main_branch)

    class1_pred = Dense(c_1, activation='softmax', name='c1_output')(main_branch)
    class2_pred = Dense(c_2, activation='softmax', name='c2_output')(main_branch)

    model = Model(inputs = inputs, outputs = [class1_pred, class2_pred])
    
    for layer in model.layers[:126]:
        layer.trainable = False
    for layer in model.layers[126:]:
        layer.trainable = True
    
    return model

In [59]:
def create_model_xception(c_1, c_2):
    base_model = tf.keras.applications.Xception(input_shape = IMG_SHAPE, 
                                                   weights = 'imagenet', 
                                                   include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)

    x_c1 = Dense(512, activation='relu', name='dense_5_c1')(x)
    x_c1 = Dropout(0.5, name='dropout_5_c1')(x_c1)
    x_c2 = Dense(512, activation='relu', name='dense_5_c2')(x)
    x_c2 = Dropout(0.5, name='dropout_5_c2')(x_c2)

    class1_pred = Dense(c_1, activation='softmax', name='c1_output')(x_c1)
    class2_pred = Dense(c_2, activation='softmax', name='c2_output')(x_c2)

    model = Model(inputs = base_model.input, outputs = [class1_pred, class2_pred])
    
    for layer in model.layers[:126]:
        layer.trainable = False
    for layer in model.layers[126:]:
        layer.trainable = True
    
    return model

In [50]:
def train_gen(training_dict, batch_size=10):
    """
    Source: https://github.com/daveboat/multitask-image-classification-keras-example
    Our image generator. This should load a batch of images of size batch_size using our training dict, resize them
    all to 200x200, and then stack them together into a (batch_size, 200, 200, 3) tensor, or a stack of (200, 200, 3)
    images
    Target is a stack of [targets_c1, targets_c2]
    Should return [image batch, target]
    """
    training_list = list(training_dict.items())
    training_len = len(training_list)
#     print('training_len = %d' % training_len)
    random.shuffle(training_list)
    list_index = 0
    current_batch_size = 0

    # yield loop
    while 1:
        images = []
        targets_c1 = []
        targets_c2 = []
        while current_batch_size < batch_size:
            images.append( image.img_to_array( image.load_img( training_list[list_index][0], target_size=img_size ) ) / 255.0 )
            targets_c1.append(training_list[list_index][1][0])
            targets_c2.append(training_list[list_index][1][1])
            list_index+=1
            current_batch_size += 1
            #print("list_index = %d, current_batch_size = %d" %(list_index, current_batch_size))
            if list_index >= training_len: list_index = 0
        current_batch_size = 0
        targets = [np.array(targets_c1), np.array(targets_c2)]

        yield [np.stack(images, axis=0)], targets

In [51]:
batch_size = 128
epochs = 10

# initialize training and validation generators
gen = train_gen(train_dict, batch_size)
val_gen = train_gen(val_dict, batch_size)
steps = len(train_dict) // batch_size
val_steps = len(val_dict) // batch_size

Try with toy dataset and try to locate issue

vanishing gradient, adjust clipping

In [58]:
# initialize and compile model
model = create_model_inception(n_labels_c1, n_labels_c2)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), #, clipvalue=0.5
              loss={'c1_output': 'sparse_categorical_crossentropy', 'c2_output': 'sparse_categorical_crossentropy'},
              loss_weights={'c1_output': 0.5, 'c2_output': 0.5},
              metrics={'c1_output': 'accuracy', 'c2_output': 'accuracy'})

# fit model
model.fit(gen, steps_per_epoch=steps, epochs=epochs, validation_data=val_gen, validation_steps=val_steps)

Epoch 1/10
103/103 [==============================] - 33s 302ms/step - loss: 7.5999 - c1_output_loss: 6.4094 - c2_output_loss: 8.7904 - c1_output_accuracy: 0.0459 - c2_output_accuracy: 0.0031 - val_loss: 6.0277 - val_c1_output_loss: 4.8751 - val_c2_output_loss: 7.1804 - val_c1_output_accuracy: 0.0452 - val_c2_output_accuracy: 5.5804e-04
Epoch 2/10
103/103 [==============================] - 30s 294ms/step - loss: 7.5963 - c1_output_loss: 6.4108 - c2_output_loss: 8.7818 - c1_output_accuracy: 0.0466 - c2_output_accuracy: 0.0026 - val_loss: 6.0263 - val_c1_output_loss: 4.8717 - val_c2_output_loss: 7.1809 - val_c1_output_accuracy: 0.0446 - val_c2_output_accuracy: 5.5804e-04
Epoch 3/10
  8/103 [=>............................] - ETA: 24s - loss: 7.5133 - c1_output_loss: 6.3695 - c2_output_loss: 8.6570 - c1_output_accuracy: 0.0449 - c2_output_accuracy: 0.0029

KeyboardInterrupt: 

In [115]:
# initialize and compile model
model2 = create_model_xception(n_labels_c1, n_labels_c2)

model2.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss={'c1_output': 'sparse_categorical_crossentropy', 'c2_output': 'sparse_categorical_crossentropy'},
              loss_weights={'c1_output': 0.5, 'c2_output': 0.5},
              metrics={'c1_output': 'accuracy', 'c2_output': 'accuracy'})

# fit model
model2.fit(gen, steps_per_epoch=steps, epochs=epochs, validation_data=val_gen, validation_steps=val_steps)

Epoch 1/10
103/103 [==============================] - 35s 321ms/step - loss: 1.6586 - c1_output_loss: 2.4315 - c2_output_loss: 0.8858 - c1_output_accuracy: 0.3339 - c2_output_accuracy: 0.8290 - val_loss: 1.6696 - val_c1_output_loss: 2.1894 - val_c2_output_loss: 1.1498 - val_c1_output_accuracy: 0.4905 - val_c2_output_accuracy: 0.9598
Epoch 2/10
103/103 [==============================] - 33s 320ms/step - loss: 0.8873 - c1_output_loss: 1.5829 - c2_output_loss: 0.1917 - c1_output_accuracy: 0.5386 - c2_output_accuracy: 0.9553 - val_loss: 0.9536 - val_c1_output_loss: 1.6430 - val_c2_output_loss: 0.2642 - val_c1_output_accuracy: 0.6055 - val_c2_output_accuracy: 0.9665
Epoch 3/10
103/103 [==============================] - 35s 339ms/step - loss: 0.6620 - c1_output_loss: 1.2024 - c2_output_loss: 0.1217 - c1_output_accuracy: 0.6404 - c2_output_accuracy: 0.9672 - val_loss: 0.6977 - val_c1_output_loss: 1.2653 - val_c2_output_loss: 0.1302 - val_c1_output_accuracy: 0.6635 - val_c2_output_accuracy: 0.

In [75]:
# Save model
model2.save("models/_Xception_multitask")

# Load model
# model = tf.keras.models.load_model('./models/_ResNet152V2model')
# model.load_weights('../Model_weights/ResNet152V2model/')


INFO:tensorflow:Assets written to: models/_Xception_multitask\assets


INFO:tensorflow:Assets written to: models/_Xception_multitask\assets


# Predict the test set
We also have to predict the real test set

In [80]:
test_dict

{'../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1878-A-2587.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4025.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4026.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4033.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4041XX.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4049.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4049A.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1880-A-4049C.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1886-A-10310.jpg': (0,
  35),
 '../Data/Rijksmuseum/output_500/test/Bolswert, Boëtius Adamsz\\RP-P-1887-A-12086.jpg': (0,
  35),
 '../Data/Rijk

In [117]:
df = []

for path, labels in test_dict.items():
    img = image.img_to_array( image.load_img( path, target_size=img_size ) ) / 255.0
    pred_c1, pred_c2 = model2.predict(np.array([img]))
    df.append([path, labels[0], labels[1], pred_c1.argmax(axis=-1)[0], pred_c2.argmax(axis=-1)[0]])

In [118]:
# df

In [119]:
df2 = pd.DataFrame(df)
df2.columns = ['path', 'cat1_true', 'cat2_true', 'cat1_pred', 'cat2_pred']
df2

,path,cat1_true,cat2_true,cat1_pred,cat2_pred
0,"../Data/Rijksmuseum/output_500/test/Bolswert, ...",0,35,10,35
1,"../Data/Rijksmuseum/output_500/test/Bolswert, ...",0,35,0,35
2,"../Data/Rijksmuseum/output_500/test/Bolswert, ...",0,35,0,35
3,"../Data/Rijksmuseum/output_500/test/Bolswert, ...",0,35,3,35
4,"../Data/Rijksmuseum/output_500/test/Bolswert, ...",0,35,0,35
...,...,...,...,...,...
3814,"../Data/Rijksmuseum/output_500/test/Vinkeles, ...",20,51,18,35
3815,"../Data/Rijksmuseum/output_500/test/Vinkeles, ...",20,51,17,35
3816,"../Data/Rijksmuseum/output_500/test/Vinkeles, ...",20,51,20,35
3817,"../Data/Rijksmuseum/output_500/test/Vinkeles, ...",20,51,13,35


In [120]:
df2['cat1_pred'].unique()

array([10,  0,  3, 18,  2,  4, 17, 13,  9,  1, 19,  8,  5,  6, 11,  7, 14,
       16, 12, 20, 15], dtype=int64)

In [96]:
# def predict_test(model, labels):
#     preds = model.predict(test_generator)
#     preds_cls_idx = preds.argmax(axis=-1)
#     idx_to_cls = {v: k for k, v in train_generator.class_indices.items()}
#     preds_cls = np.vectorize(idx_to_cls.get)(preds_cls_idx)
#     filenames_to_cls = list(zip(test_generator.filenames, preds_cls))
    
#     l = []
#     n = []
#     t = []
#     for p in filenames_to_cls:
#         n.append(p[0].split("\\")[-1][:-4])
#         l.append(p[1])
#         t.append(labels[labels['Identifier'] == p[0].split("\\")[-1][:-4]]['Creator'].values[0])
#     return pd.DataFrame(list(zip(n, l, t)), columns=['img_name', 'predicted label', 'true label'])

# res = predict_test(model2, labels)

In [ ]:
# res

In [121]:
from sklearn.metrics import accuracy_score
accuracy_c1 = accuracy_score(df2['cat1_true'], df2['cat1_pred'])
accuracy_c1

0.759884786593349

In [122]:
accuracy_c2 = accuracy_score(df2['cat2_true'], df2['cat2_pred'])
accuracy_c2

0.9753862267609322